# Primeiros passos com openMP
## 1.Aplicação
Resolução de problemas de maneira paralelizada.
Resolução de problemas com independencia de dados, bem como aceleração de problemas já existentes. 
Podemos medir seu desempenho pelo tempo de speed-up: 
- tempo de resposta 
- speed-up/processador
- Eficiencia Energética: Mflops/Watt


## 2. Primeiro contato
Programação com Threads
Suporte em multiplas plataformas e linguagens, com diferentes abstrações 
No nivel mais baixo, provido pelo SO, há primitivas para criar threads. 

API tradicional - Posix Threads(pthreads)

### Modelo de programação
- Criação de um time de threads para executar um bloco de código paralelo. 
- Time de threads pode replicar o codigo em paralelo, ou dividi-lo entre threads. 
- Threads podem ser criadas dinamicamente
- Flexibilidade em definir como variaveis do programa serão `compartilhadas` ou `replicadas` entre threads. 




*Lembre-se*:

- SIMD(Single Instruction Multiple Data) = Uma instrução executada para multiplos dados, de modo que a mesma instrução de uma unidade de controle e então executa em multiplo e diferentes dataSets. Isso permite que performe operações identicas em diferentes operandos. SIMD permite que 4, 8... 512 ? bits sejam executados no mesmo ciclo de clock. Vale lembrar tambem que SIMD usa `execution units` e nao `threads` ou `cores`.

- SIMT(Single Instruction Multiple Threads) = Bem similar com SIMD, com a diferença que SIMT faz o uso de threads, em diferentes data-set's, reduzindo a latencia que vem com busca primaria de busca. 

- SMT(Simultaneous Multi-Threading) = Permite o core da CPU rodar multiplas threads ao mesmo tempo.  Teóricamente, voce pode ter 8 threads por core via SMT, mas é viável ter somente 2.

### Hello Word from openMP

In [42]:
%%writefile 01-intro.c
#include<stdio.h>
#include<omp.h>

int main(){

  #pragma omp parallel
  { // Inicio Paralelização
    int ID = omp_get_thread_num();
    printf("Hello thread %d  ", ID);
    printf("Same thread %d\n", ID);

  } // Final paralelização
}


Overwriting 01-intro.c


In [43]:
!gcc -fopenmp 01-intro.c -o 01-intro; ./01-intro

Hello thread 0  Same thread 0
Hello thread 3  Same thread 3
Hello thread 1  Same thread 1
Hello thread 2  Same thread 2


Observe o resultado, não necessariamente o ID do thread estará alinhado com a sequencia crescente. Isso ocorre porque as threads podem ser executadas concorrentemente. e em diferentes processadores, caso existam.

### Visao Geral de OpenMP
- Modelo de multithreading de memória compartilhada. 
- Compartilhamento nao intencional de dado, que causa condições de corrida. 
- O programa de memória Compartilhada: instancia do programa -- um processo e muitas threads. Threads interagem através de leituras/escritas com o espaço de endereçamento compartilhado. 
- Escalonador SO decide quando executar cada thread(entrelaçado para ser justo). 
- Thread é basicamente uma pilha com uma parte do processo a ser executado, lembrando que variaveis globais/dinamicas podem ser acessadas por qualquer thread. 
- Threads podem ser criadas e destruidas ao longo da execução do programa, lembrando que a sincronização garante a ordem.


### Criação de Threads: Regiões Paralelas


In [69]:
%%writefile 02-array.c
#include<stdio.h>
#include<omp.h>
#include<assert.h>

void fill(int pos, float val, float* A, int N)
{
  A[pos] = val;
}

const int N = 1000;
int main(){

  float A[N]; 
  #pragma omp parallel num_threads(4)
  { 
    int ID = omp_get_thread_num();

    fill(ID, 10.8, A, N);

    

  } 
}


Overwriting 02-array.c


In [67]:
%timeit !gcc -fopenmp 02-array.c -o 02-array; ./02-array

910 ms ± 14.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Usando Laços em openMP

In [83]:
%%writefile 03-for_OMP.c
#include<stdio.h>
#include<omp.h>
#include<assert.h>


const int N = 20;
int main(){

  float A[N]; 
  float B[N]; 
  // Fill vectors
  for (int i = 0; i < N ; i++) {
    A[i] = 1.5;
    B[i] = 10.5; 
    }

  #pragma omp parallel num_threads(4)
  { //Abre um pool para "x" threads, e dividimos o trabalho entre si
    int id, i, n_threads, i_start, i_end; 
    id = omp_get_thread_num();
    n_threads = omp_get_num_threads(); 
    
    //divisao de elementos pelo numero de threads
    i_start =  id    * N / n_threads ; 
    i_start = (id+1) * N / n_threads ;

    //impede de acessar memoria indevida
    if(id == n_threads -1) i_end = N; 
    
    #pragma omp for
    for( i=i_start; i < i_end; i++){
      A[i] = A[i] + B[i];
    }     

  } 

  return 0;
}


Overwriting 03-for_loop.c


In [86]:
%%writefile 03-for_serial.c
#include<stdio.h>
#include<omp.h>
#include<assert.h>


const int N = 20;
int main()
{
  float A[N]; 
  float B[N]; 
  // Fill vectors
  for (int i = 0; i < N ; i++) {
    A[i] = 1.5;
    B[i] = 10.5; 
  }

  for(int i = 0; i < N; i++)
    A[i] = A[i] + B[i];

  return 0;
}


Writing 03-for_serial.c


In [84]:
%timeit !gcc -fopenmp 03-for_OMP.c -o 03-for_OMP; ./03-for_OMP

1.02 s ± 93.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
